In [1]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
from utils import metrics
import warnings
# 完全禁用警告
warnings.filterwarnings("ignore")

In [2]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=224,#图像尺寸
    batch_size=128,
    lr=0.0003,
    epochs=20,#epoch轮数
    hidden_size=256,
    optim="AdamW",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= (0.4914, 0.4822, 0.4465),
    std=(0.247, 0.243, 0.261),
    AMP=True,
    checkpoint_interval=0.25,#只保存4个模型
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    data_crop_rate=0.2,
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)


In [3]:
from tqdm import tqdm

torch.manual_seed(config.seed)
isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")

data\Cifar-10 数据集已存在，无需重新划分


#### 本地数据集加载

In [4]:
# def get_data_path(typ,path=config.data_path):
#     return os.path.join(path,typ)
# train_dataset=datasets.CustomImageDataset(
#     get_data_path("train"),#data/train
#     classes=config.classes,
#     transform=datasets.get_transform(
#         resize_size=config.resize_size,
#         mean=config.mean,std=config.std
#         )
# )

#### 加载CIFAR-10数据集

In [5]:

import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=config.resize_size,

    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=config.resize_size,
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


##### load数据集，并根据数据量进行裁剪。CIFAR-10较大，本身为学习项目只取其中1/4数据进行训练测试

In [6]:
crop_rate=config.data_crop_rate
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：7500
本次训练用验证集大小：2500
本次测试用测试集大小：2000
tarin_dataloader加载完毕, 58个batch, batch大小为128
val_dataloader  加载完毕, 20个batch, batch大小为128
test_dataloader 加载完毕, 16个batch, batch大小为128


#### 记录本次训练和测试用的数据量，还有数据样本信息

In [7]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(
        inputs_shape=inputs.shape
        )
    break
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    datasets_crop_rate=crop_rate,
    classes=train_val_dataset.classes,#原始数据集保留classes
)


torch.Size([128, 3, 224, 224])
torch.Size([128])


#### 加载预训练模型

In [8]:

def get_pretrained(config=config):
    '''
        获取预训练模型
        @param config: 配置文件
        @return: 预训练模型
    '''
    # model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    # model=net.AutoCNN(
    #     input_channels=3,
    #     num_classes=len(config.classes),
    #     input_size=config.inputs_shape[2:],
    #     # hidden_channels_size_1=32,
    #     # hidden_channels_size_2=64,
    #     # hidden_channels_size_3=128,
    #     # mlp_hidden_size=256
    # )
    # model=net.ResNet(
    #     num_classes=len(config.classes),
    #     input_channels=config.inputs_shape[1],
    #)#最后全局池化层压下了尺寸，不需要提供输入尺寸
    # model=net.AlexNet(
    #     num_classes=len(config.classes),
    # )
    # model=net.VGGNet(
    #     input_channels=config.inputs_shape[1],
    #     num_classes=len(config.classes),
    #     config="D",
    #     classifier_hidden_size=[4096,1024]
    # )
    # model=net.GoogLeNet(
    #     input_channels=3,
    #     num_classes=len(config.classes),
    #     AAP_shape=(4,4),
    #     aux_classify=True
    # )
    # model=net.MobileNetV1(
    #     num_classes=len(config.classes),
    #     gamma=1
    # )
    # model=torchvision.models.vision_transformer.VisionTransformer(
    #     image_size=config.inputs_shape[2:],
    #     embed_dim=343,
        
    #     depth=6,#decoder的个数
    #     num_heads=7,
    #     patch_size=(16,16),
    #     mlp_ratio=2,
    #     dropout_rate=0.2,
    #     attention_dropout_rate=0.1,
    # )
    model=net.VisionTransformer(
        image_size=config.inputs_shape[2:],
        num_classes=len(config.classes),
        embed_dim=343,
        depth=6,#decoder的个数
        num_heads=7,
        patch_size=(16,16),
        mlp_ratio=2,
        dropout_rate=0.1,
        attention_dropout_rate=0.1,
    )
    return model.to(config.device)

test_model=get_pretrained()
measurer=metrics.ModelMeasurer(test_model)
unit=1
parameters_num,inference_time=measurer.simply_check_model(input_shape=config.inputs_shape)
print(f"inference_time:{inference_time} s")
print(f"parameters_num:{parameters_num}")
config.update(
    network=test_model.__class__.__name__,
    inference_time=inference_time,
    parameters_num=parameters_num,
)
config

参数数量：6009713


Testing ...: 100%|██████████| 300/300 [00:34<00:00,  8.74it/s]

推理一个batch的时间：0.11337436167399088 s
inference_time:0.11337436167399088 s
parameters_num:6009713


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 224 
batch_size : 128 
lr : 0.0003 
epochs : 20 
hidden_size : 256 
optim : AdamW 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : (0.4914, 0.4822, 0.4465) 
std : (0.247, 0.243, 0.261) 
AMP : True 
checkpoint_interval : 0.25 
source_dir : Cifar-10 
data_path : data\Cifar-10 
data_crop_rate : 0.2 
device : cuda 
inputs_shape : torch.Size([128, 3, 224, 224]) 
train_datasize : 7500 
val_datasetsize : 2500 
test_datasetsize : 2000 
datasets_crop_rate : 0.2 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
network : VisionTransformer 
inference_time : 0.11337436167399088 
parameters_num : 6009713 

#### 开始训练

In [ ]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
elif config.optim == "AdamW":
    optimizer = optim.AdamW(model.parameters(), lr=config.lr,weight_decay=1e-4)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            model, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            checkpoint_interval=10000,
            show_progress_interval=1,
            AMP=config.AMP,#是否使用混合精度训练
            multi_loss_weight=[1,0.3,0.3],
            lr_scheduler_step=0.7,#动态调整学习率
            lr_sheduler_patience=0.1
            # num_epochs=config.epochs
        )

Training Epochs:   5%|▌         | 1/20 [00:27<08:50, 27.93s/it]

Epoch 1/20, train_Loss: 2.0731, val_Loss: 2.0112 ,Val Accuracy: 0.2320, Precision: 0.2229, Recall: 0.2372
当前最好的模型：acc : 0.2320 ,loss : 2.0731 ,precision : 0.2229 ,recall : 0.2372 ,ap : 0.1329 ,epoch : 0 


Training Epochs:  10%|█         | 2/20 [00:57<08:35, 28.65s/it]

Epoch 2/20, train_Loss: 1.9625, val_Loss: 1.9305 ,Val Accuracy: 0.2724, Precision: 0.3075, Recall: 0.2757
当前最好的模型：acc : 0.2724 ,loss : 1.9625 ,precision : 0.3075 ,recall : 0.2757 ,ap : 0.1326 ,epoch : 1 


In [ ]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

20 epoch中 最好的模型
acc : 0.2232 ,loss : 2.0785 ,precision : 0.2006 ,recall : 0.2218 ,ap : 0.1450 ,epoch : 7 


#### 保存模型超参数和训练日志

In [ ]:
saveDir=r'save_weights'
saveDir=os.path.join(
        saveDir,
        f'{bestMod.model.__class__.__name__[:10]}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,3)}-max_epochs={config.epochs}'
)  
utils.saveProcess(
    saveDir=saveDir,
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

save_weights\VisionTran-acc=0.2232-loss=2.078-max_epochs=20
{'acc': 0.2232, 'model': 'save_weights\\VisionTran-acc=0.2232-loss=2.078-max_epochs=20\\best.pth', 'loss': 2.078454231604552, 'precision': 0.20056851553880128, 'recall': 0.22175380924847415, 'ap': 0.14498184505628203, 'epoch': 7, 'checkpoints': {'checkpoint_0': 'save_weights\\VisionTran-acc=0.2232-loss=2.078-max_epochs=20\\checkpoint_0.pth'}}


#### 进行测试

In [ ]:
import torch
from utils import utils,train_val
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))
# saveDir=r'save_weights\BinaryClassificationMobileNetV3Large-acc=0.74336-loss=1.671334-max_epochs=40-1100'
Model,config,logs=utils.loadProcess(saveDir=saveDir)
metrics=train_val.validate_model(
    model=Model.model,
    val_loader=test_loader,
    device=config.device,
    only_val=True,
    criterion=criterion
)

metrics

模型测试中:: 100%|██████████| 21/21 [00:03<00:00,  5.46it/s]


{'Accuracy': 0.1885,
 'Precision': 0.1939355407747949,
 'Recall': 0.19066346467877748,
 'F1': 0.14852626435224078,
 'AP': 0.12870702866241254,
 'Loss': 2.1067892880666825}

#### 保存数据到tensorboard

In [ ]:
from utils import metrics,utils
recoder=metrics.TensorboardRecorder(#存到tensorboard显示
    log_dir="runs/",
    input_shape=[4,3,128,128],
    model=model

)
recoder.logs_scalars(
    logs.logs,
    prefix="train"
)